In [1]:
import numpy as np
import pandas as pd

In [46]:
class simplex():
    def __init__(self, A, b, c):
        self.A = A
        self.b = b
        self.c = c
        self.X, self.NormG, self.funValue = [], [], []
    # 定义二次函数
    def fun(self, x):
        return (x.T)@self.A@x/2 + self.b@x + self.c
    
    def fun_list(self, array):
        funList = []
        for k in array[:,]:
            funList.append(self.fun(k))
        return funList
    
    # 梯度
    def objGradient(self, x):
        g = self.A @ np.array(x) + self.b
        gNorm = np.linalg.norm(g, axis = 0)
        return g, gNorm
    
    # 终止条件1
    def absSum(self, funList, XL):
        funList = np.array(funList) - np.array(self.fun(XL))
        return sum(funList**2)
    
    # 终止条件2
    def relativeDiff(self, XL, XH):
        return abs((self.fun(XH)-self.fun(XL))/self.fun(XL))
    
    # 得到一个单纯性
    def get_simplex(self, initial_x):
        funList = self.fun_list(initial_x)
        XL = initial_x[funList.index(sorted(funList)[0]), :]
        XH = initial_x[funList.index(sorted(funList)[-1]), :]
        XG = initial_x[funList.index(sorted(funList)[-2]), :]
        xn_1 = (initial_x.sum(axis = 0) - XH)/3
        xn_2 = 2*xn_1 - XH
        return XL, XH, XG, xn_1, xn_2, funList
    
    # 迭代主体
    def iterate(self, initial_x, epsilon1, epsilon2, alpha, beta):
        '''
        params:
        epsilon:迭代终止界限
        initial_x:初始的n+1个点，以array形式传入
        '''
        self.X, self.NormG, self.funValue = [], [], []
        XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
        print(XL, XH, XG, xn_1, xn_2, funList)
        self.X.append(XH.copy())
        g, gNorm = self.objGradient(XH)
        self.NormG.append(gNorm)
        self.funValue.append(self.fun(XL))
        x = initial_x
        while self.absSum(funList, XL) > epsilon1 and self.relativeDiff(XH, XL) > epsilon2:
            print(XL, XH, XG, xn_1, xn_2, funList)
            if self.fun(xn_2) < self.fun(XL):
                xn_3 = xn_1 + alpha*(xn_2 - xn_1)
                if self.fun(xn_3) < self.fun(xn_2):
                    XH = xn_3
                    initial_x = np.vstack((XH, XG, XL))
                    XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XL))
                else:
                    XH = xn_2
                    initial_x = np.vstack((XH, XG, XL))
                    XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XL))
            elif self.fun(XL)<=self.fun(xn_2)<self.fun(XG):
                XH = xn_2
                initial_x = np.vstack((XH, XG, XL))
                XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
                self.X.append(XH)
                g, gNorm = self.objGradient(XH)
                self.NormG.append(gNorm)
                self.funValue.append(self.fun(XL))
            elif self.fun(XG)<=self.fun(xn_2)<self.fun(XH):
                xn_4 = xn_1 + beta*(xn_2 - xn_1)
                XH = xn_4
                initial_x = np.vstack((XH, XG, XL))
                XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
                self.X.append(XH)
                g, gNorm = self.objGradient(XH)
                self.NormG.append(gNorm)
                self.funValue.append(self.fun(XL))
            elif self.fun(xn_2)>=self.fun(XH):
                xn_5 = xn_1 + beta*(XH - xn_1)
                if self.fun(xn_5) >= self.fun(XH):
                    for k in range(len(x)):
                        x[k,:] = (x[k,:] + XL)/2
                    XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(x)
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XL))
                else:
                    XH = xn_5
                    initial_x = np.vstack((XH, XG, XL))
                    XL, XH, XG, xn_1, xn_2, funList = self.get_simplex(initial_x)
                    self.X.append(XH)
                    g, gNorm = self.objGradient(XH)
                    self.NormG.append(gNorm)
                    self.funValue.append(self.fun(XL))
        return self.X, self.NormG, self.funValue

In [47]:
A = np.array([[2, 0], [0, 4]])
b = np.array([-4, -8])
c = 5
epsilon = 0.1
eighth = simplex(A, b, c)

In [48]:
x = np.array([[0, 0], [0.965, 0.259], [0.259, 0.965]])
X, NormG, funValue = eighth.iterate(initial_x=x, epsilon1=0.1, epsilon2=0.1, alpha=1.1, beta=0.5)

[0.259 0.965] [0. 0.] [0.965 0.259] [0.408 0.408] [0.816 0.816] [5.0, 0.13338699999999903, -1.8264690000000003]
[0.259 0.965] [0. 0.] [0.965 0.259] [0.408 0.408] [0.816 0.816] [5.0, 0.13338699999999903, -1.8264690000000003]
[0.8568 0.8568] [0.965 0.259] [0.259 0.965] [0.37193333 0.60726667] [-0.22113333  0.95553333] [-3.07928128, 0.13338699999999903, -1.8264690000000003]
[0.8568 0.8568] [0.0754 0.7814] [0.259 0.965] [0.37193333 0.60726667] [0.66846667 0.43313333] [-0.325942920000001, -1.8264690000000003, -3.07928128]
[0.8568 0.8568] [0.22366667 0.69433333] [0.259 0.965] [0.37193333 0.60726667] [0.5202 0.5202] [-0.43510900000000063, -1.8264690000000003, -3.07928128]
[0.8568 0.8568] [0.2978 0.6508] [0.259 0.965] [0.37193333 0.60726667] [0.44606667 0.56373333] [-0.46183388000000036, -1.8264690000000003, -3.07928128]
[0.8568 0.8568] [0.33486667 0.62903333] [0.259 0.965] [0.37193333 0.60726667] [0.409  0.5855] [-0.46823178, -1.8264690000000003, -3.07928128]
[0.8568 0.8568] [0.3534  0.61815]

In [49]:
res8 = pd.DataFrame({'迭代XH值':X, '梯度范数':NormG, '函数值':funValue})

In [50]:
res8

,迭代XH值,梯度范数,函数值
0,"[0.0, 0.0]",8.944272,-1.826469
1,"[0.965, 0.259]",7.265136,-3.079281
2,"[0.07540000000000008, 0.7814000000000001]",6.210967,-3.079281
3,"[0.22366666666666668, 0.6943333333333335]",6.316462,-3.079281
4,"[0.29779999999999995, 0.6508]",6.380861,-3.079281
5,"[0.33486666666666665, 0.6290333333333333]",6.415876,-3.079281
6,"[0.35340000000000005, 0.61815]",6.434075,-3.079281
7,"[0.3626666666666667, 0.6127083333333332]",6.443345,-3.079281
8,"[0.36730000000000007, 0.6099874999999999]",6.448022,-3.079281
9,"[0.54980625, 0.88074375]",7.028409,-2.548070
